# ELGA

## 1. Data pre processing 

In [2]:
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.decomposition import PCA, KernelPCA
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import SVR, LinearSVR
from sklearn.linear_model import ElasticNet, SGDRegressor, BayesianRidge
from sklearn.kernel_ridge import KernelRidge
from xgboost import XGBRegressor
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
data = pd.read_csv('data1.csv')
y = data['Roughness']
X = data.drop('Roughness', axis = 1)
X_train, X_test, y_train, y_test =train_test_split(X,y,test_size=0.1,random_state=100)
poly = PolynomialFeatures(2,include_bias=True)
Xtrans=poly.fit_transform(X_train)
scaler = MinMaxScaler()
X_scaled = scaler.fit(Xtrans).transform(Xtrans)
scalery =MinMaxScaler()
y_scaled=scalery.fit(np.array(y_train).reshape(-1, 1)).transform(np.array(y_train).reshape(-1, 1)).ravel()
Xtrans_test=poly.transform(X_test)
X_scaled_test = scaler.transform(Xtrans_test)
y_scaled_test=scalery.transform(np.array(y_test).reshape(-1, 1)).ravel()

### hyperparameter cross-validation

In [ ]:
class grid():
    def __init__(self,model):
        self.model = model
    
    def grid_get(self,X,y,param_grid):
        grid_search = GridSearchCV(self.model,param_grid,cv=5, scoring="neg_mean_absolute_error")
        grid_search.fit(X,y)
        print(grid_search.best_params_, -grid_search.best_score_)
        grid_search.cv_results_['mean_test_score'] = -grid_search.cv_results_['mean_test_score']
        print(pd.DataFrame(grid_search.cv_results_)[['params','mean_test_score','std_test_score']])

grid(Ridge()).grid_get(X_scaled, y_scaled,{"alpha":[0.8,0.7,0.9,1,1.5]})
grid(Lasso()).grid_get(X_scaled, y_scaled,{"alpha":[0.01,0.02,0.03],"max_iter":[100,200,300]})
grid(RandomForestRegressor()).grid_get(X_scaled, y_scaled,{"random_state":[140,150,160],"min_samples_split":[2,3,4,5,6,7,8],"n_estimators":[7,6,5]})
grid(ElasticNet()).grid_get(X_scaled, y_scaled,{"alpha":[0.001],"max_iter":[100]})
grid(XGBRegressor(objective='reg:squarederror')).grid_get(X_scaled, y_scaled,{"max_depth":[4],"learning_rate":[0.04],"n_estimators":[1300,1600,1500,1400,1700]})

## 2. Multi Machine learning regression model

In [ ]:
rd=Ridge(alpha=0.9)
las=Lasso(alpha=0.01,max_iter=10000,tol=0.001)
rf=RandomForestRegressor(random_state=150,min_samples_split=5,n_estimators=6)
ela=ElasticNet(tol=0.01,alpha=0.01,max_iter=100)
xgb=XGBRegressor(objective='reg:squarederror')

In [ ]:
def rmse_cv(model,X,y):
    rmse = -cross_val_score(model, X, y, scoring="neg_mean_absolute_error", cv=5)
    return rmse

In [ ]:
models = [rd,las,rf,ela,xgb]
names = ["rd", "las", "rf", "ela", "xgb"]

In [ ]:
for name, model in zip(names, models):
    score = rmse_cv(model, X_scaled, y_scaled)
    print("{}: {:.6f}, {:.4f}".format(name,score.mean(),score.std()))

In [ ]:
for name, model in zip(names, models):
    model.fit(X_scaled,y_scaled)
    pred=model.predict(X_scaled_test)
    ypred=scalery.inverse_transform(pred.reshape(-1, 1)).ravel()
    print(ypred) ## used in GA
    mean=np.mean(np.abs((y_test-ypred)/ypred))
    print(name,mean)

## 3. GA

In [ ]:
import numpy as np
import geatpy as ea
from aimfunc import aimfunc
import time

x1=[0,1]
x2=[0,1]
x3=[0,1]
x4=[0,1]
x5=[0,1]

b1=[1,1]
b2=[1,1]
b3=[1,1]
b4=[1,1]
b5=[1,1]

ranges=np.vstack([x1,x2,x3,x4,x5]).T
borders=np.vstack([b1,b2,b3,b4,b5]).T
varTypes=np.array([0,0,0,0,0])
Encoding='BG'
codes=[0,0,0,0,0]
precisions=[2,2,2,2,2]
scales=[0,0,0,0,0]
FieldD=ea.crtfld(Encoding,varTypes,ranges,borders,precisions,codes,scales)
NIND      = 10000;
MAXGEN    = 100;
maxormins = [1] 
maxormins = np.array(maxormins)
selectStyle = 'rws' 
recStyle  = 'xovdp' 
mutStyle ='mutbin'

pc=0.9
Lind=int(np.sum(FieldD[0,:]))
pm=1/Lind
obj_trace=np.zeros((MAXGEN,2))
var_trace=np.zeros((MAXGEN,Lind))
start_time=time.time()
Chrom=ea.crtpc(Encoding,NIND,FieldD)
variable=ea.bs2real(Chrom,FieldD)

ObjV,CV=aimfunc(variable)
FitnV = ea.ranking(ObjV,CV,maxormins)
best_ind = np.argmax(FitnV)
#
for gen in range(MAXGEN):
    SelCh = Chrom[ea.selecting(selectStyle,FitnV,NIND-1),:] 
    SelCh = ea.recombin(recStyle, SelCh, pc) 
    SelCh = ea.mutate(mutStyle, Encoding, SelCh, pm)
    Chrom = np.vstack([Chrom[best_ind, :], SelCh])
    Phen = ea.bs2real(Chrom, FieldD)
    ObjV, CV = aimfunc(Phen)
    FitnV = ea.ranking(ObjV, CV,maxormins)
    best_ind = np.argmax(FitnV) 
    obj_trace[gen,0]=np.sum(ObjV)/ObjV.shape[0] 
    obj_trace[gen,1]=ObjV[best_ind]
    var_trace[gen,:]=Chrom[best_ind,:]

end_time = time.time()
ea.trcplot(obj_trace, [['individuals mean objective value ', 'best individual objective value']])
"""============================result============================"""
best_gen = np.argmax(obj_trace[:, [1]])
print('optimal solution objective value：', obj_trace[best_gen, 1])
variable =ea.bs2real(var_trace[[best_gen], :], FieldD)
# variable = var_trace[[best_gen], :] 
print('optimal parameter value：')
for i in range(variable.shape[1]):
    print('x'+str(i)+'=',variable[0, i])
print('time：', end_time - start_time, 's')

In [ ]:
import numpy as np

def aimfunc(Phen):
    x1 = Phen[:, [0]] 
    x2 = Phen[:, [1]]
    x3 = Phen[:, [2]]
    x4 = Phen[:, [3]]
    x5 = Phen[:, [4]]
    # pred is the predictive value predicted by each model 
    pred = np.array([[74,166,107,70,60],
                     [73,125,97,69,66],
                     [68,120,70,71,76],
                     [79,120,105,68,64],
                     [98,140,89,43,45]])
    test = np.array([44,144,88,66,47])
    predy=pred[0] * x1 + pred[1] * x2 + pred[2] * x3 + pred[3] * x4 + pred[4] * x5
    ObjV = np.mean(np.abs(predy - test) / predy,axis=1,keepdims=True)
    CV = np.abs(x1 + x2 + x3 + x4 + x5 - 1)
    return [ObjV,CV]


## 4. Ensemble model

In [ ]:
class AverageWeight(BaseEstimator, RegressorMixin):
    def __init__(self,mod,weight):
        self.mod = mod
        self.weight = weight
        
    def fit(self,X,y):
        self.models_ = [clone(x) for x in self.mod]
        for model in self.models_:
            model.fit(X,y)
        return self
    
    def predict(self,X):
        w = list()
        pred = np.array([model.predict(X) for model in self.models_])
        print(scalery.inverse_transform(np.array(pred).reshape(-1, 1)).ravel())
        
        # for every data point, single model prediction times weight, then add them together
        for data in range(pred.shape[1]):
            single = [pred[model,data]*weight for model,weight in zip(range(pred.shape[0]),self.weight)]
            w.append(np.sum(single))
        return w

In [ ]:
#ELGA 
#weight comes from the result of GA
weight_avg = AverageWeight(mod = models,weight=weight)
weight_avg.fit(X_scaled,y_scaled)
pre=weight_avg.predict(X_scaled_test)
Avgpred=scalery.inverse_transform(np.array(pre).reshape(-1, 1)).ravel()
np.mean(np.abs((y_test-Avgpred)/y_test))

# ANN

In [ ]:
import tensorflow as tf
import numpy  as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn import model_selection
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers,initializers,optimizers,losses
import tensorflow.keras.backend as K

In [ ]:
df=pd.read_csv('data1.csv')
y=df['Roughness']
X=df.drop('Roughness',axis=1)

In [ ]:
M1 = MinMaxScaler()
X_scaled = M1.fit_transform(X)
M2 = MinMaxScaler()
y_scaled = M2.fit_transform(np.array(y).reshape(-1,1))

In [ ]:
X_train,X_valid,y_train,y_valid=model_selection.train_test_split(X_scaled,y_scaled,test_size=0.1,random_state=10)

In [ ]:
def mean_pred(y_true, y_pred):
    return K.mean(tf.abs(y_pred-y_true)/y_true)

In [ ]:
#Model Built
inputs=keras.Input(shape=(6,))
x=layers.Dense(4,activation='tanh')(inputs)
x=layers.Dense(4,activation='tanh')(x)
x=layers.Dense(4,activation='tanh')(x)
x=layers.Dense(4,activation='tanh')(x)
outputs=layers.Dense(1,activation='linear')(x)

model=keras.Model(inputs=inputs,outputs=outputs)
model.compile(loss='mean_absolute_error',optimizer='adam',metrics=[mean_pred])

history=model.fit(X_train,y_train,batch_size=1,epochs=100)

In [ ]:
# prediction
pred=model.predict(X_valid)
ypred=M2.inverse_transform(pred.reshape(-1, 1)).ravel()
yvalid=M2.inverse_transform(y_valid.reshape(-1, 1)).ravel()
np.mean(np.abs(yvalid-ypred)/ypred)

In [ ]:
# plot the loss and mean_pred
loss = history.history.get('loss')
mean_pred = history.history.get('mean_pred')
val_loss = history.history.get('val_loss')
val_mean_pred = history.history.get('val_mean_pred')

plt.subplot(1, 2, 1)
plt.plot(loss)
plt.plot(val_loss)
plt.xlabel("Epochs")
plt.ylabel("loss")
plt.legend(['loss','val_loss'])

plt.subplot(1, 2, 2)
plt.plot(mean_pred)
plt.plot(val_mean_pred)
plt.xlabel("Epochs")
plt.ylabel("mean_pred")
plt.legend(['mean_pred','val_mean_pred'])